In [120]:

import json
import csv
import torch
from torch.utils.data import Dataset,DataLoader

from loguru import logger
import pytorch_lightning as pl

from torchnlp.utils import collate_tensors, lengths_to_mask

from transformers import AutoTokenizer



def load_mednli(datadir='../data/mednli/'):
    filenames = [
        'mli_train_v1.jsonl',
        'mli_dev_v1.jsonl',
        'mli_test_v1.jsonl',
    ]

    filenames = [datadir+f  for f in filenames]

    mednli_train, mednli_dev, mednli_test = [read_mednli(f) for f in filenames]

    return mednli_train, mednli_dev, mednli_test


def read_mednli(filename) -> list:
    data = []

    with open(filename, 'r') as f:
        for line in f:
            example = json.loads(line)

            premise = (example['sentence1'])
            hypothesis = (example['sentence2'])
            label = example.get('gold_label', None)
            data.append((premise,hypothesis,label))

    print(f'MedNLI file loaded: {filename}, {len(data)} examples')
    return data


In [115]:
dm = MedNLIDataModule(None)
dm.setup()
test_dl = dm.test_dataloader()
test_dl

MedNLI file loaded: ../data/mednli/mli_train_v1.jsonl, 11232 examples
MedNLI file loaded: ../data/mednli/mli_dev_v1.jsonl, 1395 examples
MedNLI file loaded: ../data/mednli/mli_test_v1.jsonl, 1422 examples

Downloading: 100%|██████████| 442/442 [00:00<00:00, 205kB/s]
2020-11-27 10:32:37.008 | INFO     | __main__:setup:76 - MedNLI JSONs loaded...
2020-11-27 10:32:37.010 | WARNING  | __main__:test_dataloader:93 - Loading testing data...


In [116]:
batch = next(iter(test_dl))
input, target = batch


In [119]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
model = AutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased')


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

In [45]:
encoded_dataset = test.map(preprocess_function, batched=True)
encoded_dataset

AttributeError: 'list' object has no attribute 'map'